In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
import os

In [5]:
# Check out previously cleaned data
path = '..\\cleaned_scripts\\aladdin.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,chars,lines,line_num,scene_setup,mod_lines,line_stemmed,scene_setup_stemmed
0,0,ALADDIN,THE COMPLETE SCRIPT\r\nCOMPILED BY BEN SCRIP...,1,(Portions Copyright (c),THE COMPLETE SCRIPT\r\nCOMPILED BY BEN SCRIP...,complet script compil ben scripp rqnpq cmuvm c...,portion copyright c
1,1,PEDDLER,Oh I come from a land\r\n From a farawa...,5,(Camera zooms in hitting\r\n peddler in face),Oh I come from a land\r\n From a farawa...,oh come land faraway place caravan camel roam ...,camera zoom hit peddler face
2,2,JAFAR,You...are late.\r\n,65,NaN,You...are late.,late,NaN
3,3,GAZEEM,"A thousand apologies, O patient one.\r\n",66,NaN,"A thousand apologies, O patient one.",thousand apolog patient one,NaN
4,4,JAFAR,"You have it, then?\r\n",67,NaN,"You have it, then?",NaN,NaN


In [6]:
# Read in Mulan
path = '..\\scripts\\mulan.txt'
file = open(path, 'r')
script = file.readlines()
file.close()

script = script[5:]
script[:5]

['***\n',
 '\n',
 'A Chinese painting of the Great Wall of China is slowly painted across the screen. The words Walt Disney Pictures present and then Mulan and the dragon symbol appear. The painting becomes animated, and a Chinese soldier walks around his post. Suddenly, a bird of prey hits the back of his head ...\n',
 'Soldier:\n',
 'Ah! [The hawk lets out a call. The soldier walks over to the edge of the wall. A hook comes flying up, followed by many more.]\n']

In [8]:
# 1) read in the scripts: character, line
path = "..\\scripts\\mulan.txt"
file = open(path, 'r')
script = file.readlines()
file.close()
start, end = 8, 1231


line_count = 0 # number of lines total 
characters = set() # all the characters
lines = [] # character with their lines, index are the line numbers
line_nums = [] # actual line numbers in the script


# helper functions

# Only include what's outside of bracket
def remove_setup(string): 
    stack = False
    new_str = ""
    for s in string:
        if s=='[':
            stack = True
        elif s==']':
            stack = False
        else: 
            if not stack:
                new_str += s
    return new_str.strip() 

# Only include bracket
def return_setup(string): 
    stack = False
    new_str = ""
    for s in string:
        if s=='[':
            stack = True
        elif s==']':
            stack = False
        else: 
            if stack:
                new_str += s
    return new_str.strip() 

def remove_linebreak(string):
    return string.replace('\n', ' ')

# iterate through the lines 
for i in range(start, end+1):
    holder = script[i] # line that is being read
    if ':' in holder and 'this:'not in holder:
        character = holder[:-2]
        characters.add(character) # record for unique characters
        lines.append([character, ""])
        line_count += 1
        line_nums.append(i)
    else:
        if len(holder) != 0 and line_count != 0:
            lines[line_count-1][1] += holder

lines = np.array(lines)

# # 2) put the lines into a df and store it 
# example: 
d = {
        'chars':lines[:,0],
        'lines': lines[:,1], 
    }
scripts = pd.DataFrame(d)

scripts['line_num'] = pd.Series(line_nums)
scripts['mod_line'] = scripts.apply(lambda x: remove_linebreak(remove_setup(x.iloc[1])), axis=1)
scripts['scene_setup'] = scripts.apply(lambda x: remove_linebreak(return_setup(x.iloc[1])), axis=1)

out_path = os.path.join('..\cleaned_scripts', 'mulan.csv')
scripts.to_csv(out_path)

In [9]:
in_path = os.path.join('..\cleaned_scripts', 'mulan.csv')
df = pd.read_csv(in_path)
df.head()

,Unnamed: 0,chars,lines,line_num,mod_line,scene_setup
0,0,Soldier,Ah! [The hawk lets out a call. The soldier wal...,8,Ah!,The hawk lets out a call. The soldier walks ov...
1,1,Soldier,[Running back to his post] We're under attack!...,10,We're under attack! Light the signal! The door...,Running back to his post
2,2,Soldier,"[lighting the signal with a torch, while stari...",13,Now all of China knows you're here.,"lighting the signal with a torch, while starin..."
3,3,Shan Yu,[picking up a flag and toasting it in the sign...,15,Perfect. *** The Chinese General and two sold...,picking up a flag and toasting it in the signa...
4,4,General,"Your Majesty, the Huns have crossed our Northe...",20,"Your Majesty, the Huns have crossed our Northe...",NaN


In [11]:
import nltk
# nltk.download([
# "names",
# "stopwords",
# "state_union",
# "twitter_samples",
# "movie_reviews",
# # "averaged_perceptron_tagger",
# "vader_lexicon",
# "punkt"])

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(df['mod_line'][1])['compound']

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\book2/nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\lib\\nltk_data'
    - 'C:\\Users\\book2\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [13]:
df['mod_line'][1]

"We're under attack! Light the signal! The door opens, revealing a couple of Huns. The soldier climbs up the ladder."

In [14]:
def get_emotion(line):
    if type(line) == float:
        return
    sia = SentimentIntensityAnalyzer()
    diction = sia.polarity_scores(line)
    return diction['compound']

In [16]:
df['mod_line'].values[:5]

array(['Ah!',
       "We're under attack! Light the signal! The door opens, revealing a couple of Huns. The soldier climbs up the ladder.",
       "Now all of China knows you're here.",
       "Perfect. ***  The Chinese General and two soldiers walk into the Emperor's chamber and bow.",
       'Your Majesty, the Huns have crossed our Northern Border.'],
      dtype=object)

In [20]:
df['emotion_score'] = df['mod_line'].apply(get_emotion)

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\book2/nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Miniconda3\\lib\\nltk_data'
    - 'C:\\Users\\book2\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [19]:
df.head()

,Unnamed: 0,chars,lines,line_num,mod_line,scene_setup
0,0,Soldier,Ah! [The hawk lets out a call. The soldier wal...,8,Ah!,The hawk lets out a call. The soldier walks ov...
1,1,Soldier,[Running back to his post] We're under attack!...,10,We're under attack! Light the signal! The door...,Running back to his post
2,2,Soldier,"[lighting the signal with a torch, while stari...",13,Now all of China knows you're here.,"lighting the signal with a torch, while starin..."
3,3,Shan Yu,[picking up a flag and toasting it in the sign...,15,Perfect. *** The Chinese General and two sold...,picking up a flag and toasting it in the signa...
4,4,General,"Your Majesty, the Huns have crossed our Northe...",20,"Your Majesty, the Huns have crossed our Northe...",NaN
...,...,...,...,...,...,...
481,481,Mushu,"Take it, Crickee! [Crickee plays a set of drum...",1222,"Take it, Crickee!","Crickee plays a set of drums, and all the ance..."
482,482,Ancestor #2,"You know, she gets it from my side of the fami...",1224,"You know, she gets it from my side of the fami...",NaN
483,483,Great Ancestor,Guardians.\n,1226,Guardians.,NaN
484,484,Mulan,"Thanks, Mushu. [She kisses him on the forehead...",1228,"Thanks, Mushu.","She kisses him on the forehead. Suddenly, Litt..."


In [21]:
g = sns.lineplot(x = df.index, y = df['emotion_score'], hue = df['chars'])
g.legend_.remove()

KeyError: 'emotion_score'

In [70]:
mushu = df[(df['chars'] == 'Mushu') ]

In [71]:
mushu

,Unnamed: 0,chars,lines,line_num,mod_line,scene_setup,emotion_score
104,104,Mushu,"I live! So, tell me, what mortal needs my prot...",316,"I live! So, tell me, what mortal needs my prot...",NaN,0.6588
106,106,Mushu,"And lemme say something, anyone who's foolish ...",320,"And lemme say something, anyone who's foolish ...",NaN,-0.6114
108,108,Mushu,Protect the family.\n,324,Protect the family.,NaN,0.3818
110,110,Mushu,I ring the gong.\n,328,I ring the gong.,NaN,0.0000
112,112,Mushu,"One family reunion coming right up. Okay, peop...",332,"One family reunion coming right up. Okay, peop...",NaN,0.8580
...,...,...,...,...,...,...,...
442,442,Mushu,Ahahahahahaha! [He catches Crickee] You are a ...,1138,Ahahahahahaha! You are a lucky bug!,He catches Crickee,0.5242
450,450,Mushu,Our little baby is all grown up and saving Chi...,1154,Our little baby is all grown up and saving Chi...,To Crickee,0.0000
478,478,Mushu,"[to Great Ancestor] Who did a good job? C'mon,...",1215,"Who did a good job? C'mon, tell me who did a g...",to Great Ancestor,0.7003
480,480,Mushu,AAAAAHHH-HAAAAAAAA!!! Whoohoohoohoohoooo!\nCri...,1219,AAAAAHHH-HAAAAAAAA!!! Whoohoohoohoohoooo! Cric...,NaN,0.0000


In [ ]:
g = sns.lineplot(x = mushu.index, y = mushu['emotion_score'])

In [23]:
from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))

Downloading: 100%|██████████| 629/629 [00:00<00:00, 655kB/s]
Downloading:   2%|▏         | 5.00M/268M [01:17<3:24:50, 21.4kB/s]WARNING: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='pypi.org', port=443): Read timed out. (read timeout=15)")': /simple/transformers/
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\miniconda3\\lib\\site-packages\\transformers-4.6.1-py3.8.egg'
Consider using the `--user` option or check the permissions.

  Attempting uninstall: transformers
    Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6.1:
Downloading:   4%|▎         | 9.57M/268M [11:10<15:16:23, 4.70kB/s]

KeyboardInterrupt: 

In [1]:
pip install git+https://github.com/huggingface/transformers

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers==3.0.0

ERROR: Command errored out with exit status 128: git clone -q https://github.com/huggingface/transformers 'C:\Users\wanga\AppData\Local\Temp\pip-req-build-2ekc5kdm' Check the logs for full command output.


  Cloning https://github.com/huggingface/transformers to c:\users\wanga\appdata\local\temp\pip-req-build-2ekc5kdm
^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install https://github.com/google/sentencepiece/releases/download/v0.1.84/sentencepiece-0.1.84-cp38-cp38-manylinux1_x86_64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: sentencepiece-0.1.84-cp38-cp38-manylinux1_x86_64.whl is not a supported wheel on this platform.
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\wanga\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 437, in _error_catcher
    yield
  File "C:\Users\wanga\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\wanga\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\wanga\anaconda3\lib\http\client.py", line 458, in read
    n = self.readinto(b)
  File "C:\Users\wanga\anaconda3\lib\http\client.py", line 502, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\wanga\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\wanga\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File 

In [ ]:
from transformers import pipeline; print(pipeline('sentiment-analysis')('I hate you'))